<a href="https://colab.research.google.com/github/mohammadp1001/kaggle_payground_prediction_s5e6/blob/main/S5E6_XGBoost_0601.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()

In [4]:
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

playground_series_s5e6_path = kagglehub.competition_download('jigsaw-agile-community-rules')
print('Data source import complete.')

Data source import complete.


In [5]:
import os
import numpy as np
import torch
from tqdm import tqdm
import pandas as pd
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

In [6]:
train = pd.read_csv("/kaggle/input/jigsaw-agile-community-rules/train.csv")
test = pd.read_csv("/kaggle/input/jigsaw-agile-community-rules/test.csv")

In [7]:
train.head(2)

,row_id,body,rule,subreddit,positive_example_1,positive_example_2,negative_example_1,negative_example_2,rule_violation
0,0,Banks don't want you to know this! Click here ...,"No Advertising: Spam, referral links, unsolici...",Futurology,If you could tell your younger self something ...,hunt for lady for jack off in neighbourhood ht...,Watch Golden Globe Awards 2017 Live Online in ...,"DOUBLE CEE x BANDS EPPS - ""BIRDS""\n\nDOWNLOAD/...",0
1,1,SD Stream [ ENG Link 1] (http://www.sportsstre...,"No Advertising: Spam, referral links, unsolici...",soccerstreams,[I wanna kiss you all over! Stunning!](http://...,LOLGA.COM is One of the First Professional Onl...,#Rapper \n🚨Straight Outta Cross Keys SC 🚨YouTu...,[15 Amazing Hidden Features Of Google Search Y...,0


In [8]:
train["rule_violation"].value_counts()

rule_violation
1    1031
0     998
Name: count, dtype: int64

In [9]:
test.head(2)

,row_id,body,rule,subreddit,positive_example_1,positive_example_2,negative_example_1,negative_example_2
0,2029,NEW RAP GROUP 17. CHECK US OUT https://soundcl...,"No Advertising: Spam, referral links, unsolici...",hiphopheads,"Hey, guys, just wanted to drop in and invite y...",Cum Swallowing Hottie Katrina Kaif Cartoon Xvi...,SD Stream Eng - [Chelsea TV USA](http://soccer...,HD Streams: |[ENG HD Stoke vs Manchester Unite...
1,2030,Make your life comfortable. Get up to 15% Disc...,No legal advice: Do not offer or request legal...,AskReddit,Get a lawyer and get the security camera foota...,That isn't drastic. You tried reaching out to ...,So what are you going to do with the insurance...,It's just for Austria & Germany. If you still ...


In [30]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('Qwen/Qwen-7B', trust_remote_code=True, pad_token='<|endoftext|>')

In [41]:
class CommentDataset(Dataset):
    """Custom Dataset for loading comments and their labels."""
    def __init__(self, tokensizer, df, max_length=1024):
        """Initialize the dataset with texts and labels.

        :param tokenizer: Tokenizer to convert text to tokens.
        :param df: DataFrame containing the data.
        :param max_length: Maximum length for tokenization. Default is 1024.
        """
        self.tokenizer = tokensizer
        self.max_length = max_length 
        self.texts = [f"Body: {row.body} Rule: {row.rule} Positive Examples: {row.positive_example_1} {row.positive_example_2} Negative Examples: {row.negative_example_1}, {row.negative_example_2}" 
                        for row in df.itertuples(index=False)]
        
        # Batch encode once during initialization
        self.encodings = self.tokenizer(
            self.texts,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )
        self.labels = torch.tensor(df['rule_violation'].values, dtype=torch.long)
        
    def __len__(self):
        return len(self.encodings['input_ids'])
    
    def __getitem__(self, idx):
        return (
            self.encodings['input_ids'][idx],
            self.labels[idx] 
            )


In [42]:
train_dataset = CommentDataset(tokenizer, train)
batch_size = 8
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)

In [43]:
for input_ids, labels in train_loader:
    print("Input IDs shape:", input_ids.shape)
    print("Labels shape:", labels.shape)
    break

Input IDs shape: torch.Size([8, 1024])
Labels shape: torch.Size([8])


As we can see the train_loader gives back input_ids and labels which contains 8 examples.